# Automatic Sandpit Refinement for D-Flow FM

This notebook provides an automated workflow for refining unstructured grids around sandpit areas in D-Flow FM models. The refinement process uses Casulli refinement to gradually transition from coarse background resolution to fine target resolution.

## Workflow Overview

1. **Configuration**: Set file paths and refinement parameters
2. **Load Grid and Polygons**: Load the grid and create/load sandpit polygons
3. **Plan Refinement**: Analyze grid resolution and generate refinement zones
4. **Execute Refinement**: Apply Casulli refinement to the grid
5. **Monitor Quality** (Optional): Analyze grid quality metrics

## Requirements

- `dfm_tools`
- `meshkernel`
- `numpy`
- `matplotlib`
- `shapely`

In [ ]:
# Step 0: Configuration and Setup
import sys
import os

# Add project root to Python path (for local installations)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import numpy as np
import dfm_tools as dfmt
import matplotlib.pyplot as plt
import xugrid as xu
import xarray as xr
import psutil
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial.distance import cdist
import scipy.interpolate
import meshkernel
import re

# Import utility functions
from src.polygon_utils import InteractivePolygonDrawer, load_pol_file, save_pol_file, generate_refinement_polygons, expand_polygon_outward, add_shape_to_polygon
from src.refinement_utils import compute_refinement_steps, apply_casulli_refinement, print_refinement_summary
from src.visualization_utils import plot_grid, is_codespace
from src.monitoring_utils import analyze_grid_quality, plot_grid_quality
from src.restart_utils import create_restart_file, compare_restart_files, regrid_restart_data
from src.netcdf_utils import export_refined_grid

# INPUT: Filenames
nc_file ='dcsm_0_125nm_2ref_bathygr7_RGFGRID_net.nc' #'DCSM-FM_0_5nm_grid_20220310_depth_20220517_net.nc'
restart_file = 'DCSM-FM_0_5nm_merged_20190127_000000_rst.nc'

# INPUT: Toggle options
visualization_bool = False      # Set to True to enable visualization
monitor_quality = False         # Set to True if you to analyze the quality of the grid
use_existing_pol_file = True    # Set to True to use an existing polygon file (no new polygons will be created)
plot_bathymetry = False           # Set to True to plot bathymetry in the background of interactive plot

# INPUT: Refinement parameters
target_resolution = 100  # Target resolution in meters for the finest refinement level
buffer_around_sandpit = 250  # Buffer around sandpit polygons in meters
N = 6  # Number of transition cells

# INPUT: Digging parameters
dig_depth = 5.0        # meters to lower bed level
slope = 0.03            # slope ratio for smooth transitions (default: 0.1)

# Auto-detect correct data path based on current working directory
if os.path.basename(os.getcwd()) == 'notebooks':
   # Running from notebooks directory (Codespace)
   data_path = os.path.join('..', 'data')
else:
   # Running from project root (local Jupyter)
   data_path = 'data'

# INPUT: File paths (input & output)
nc_path = os.path.join(data_path, 'input', nc_file)
restart_path = os.path.join(data_path, 'input', restart_file)
input_pol_file = os.path.join(data_path, 'input', 'sandpits.pol')
output_pol_file = os.path.join(data_path, 'output', 'sandpits.pol')
xyz_output_path = os.path.join(data_path, 'output', 'bathymetry.xyz')
output_dir = os.path.join(data_path, 'output')
os.makedirs(output_dir, exist_ok=True)

# Environment detection
if is_codespace():
    print("🔧 Running in GitHub Codespace")
else:
    print("🔧 Running locally")

print("✅ Configuration loaded")
print(f"   Target: {target_resolution}m | Buffer: {buffer_around_sandpit}m | Transitions: {N}")

## Step 1: Load Grid and Define Sandpit Polygons

This step loads the D-Flow FM grid and either:
- Loads existing sandpit polygons from a .pol file
- Opens an interactive drawing tool to create new polygons

The polygons define the areas where refinement will be applied.

**Note**: Interactive polygon drawing is only available when running locally, not in Codespace.

In [ ]:
# Step 1: Load Grid and Create/Load Polygons

# Check if NetCDF file exists
if not os.path.exists(nc_path):
    print(f"❌ NetCDF file not found: {nc_path}")
    if is_codespace():
        print("   File should be automatically available in Codespace")
    else:
        print("   Please download the file from: https://github.com/your-repo/releases")
        print("   Or use git lfs pull to download LFS files")
    raise FileNotFoundError(f"Required file not found: {nc_path}")

# Load grid data
print("📊 Loading grid...")
ugrid = dfmt.open_partitioned_dataset(nc_path)
ugrid_original = dfmt.open_partitioned_dataset(nc_path)  # Keep original for later comparison

# Convert to meshkernel objects for consistent plotting and refinement
mk_object = ugrid.grid.meshkernel
mk_backup = ugrid_original.grid.meshkernel  # Keep backup

# Handle polygon input/creation
if use_existing_pol_file:
    # Load existing polygon file
    if os.path.exists(input_pol_file):
        polygons = load_pol_file(input_pol_file)
        print(f"✅ Loaded {len(polygons)} sandpit polygons from file")
    else:
        print(f"❌ File {input_pol_file} not found, switching to interactive mode")
        use_existing_pol_file = False

if not use_existing_pol_file:
    # Interactive polygon creation
    if is_codespace():
        print("❌ Interactive polygon drawing not available in Codespace")
        print("   Please create a sandpits.pol file or run locally for interactive drawing")
        raise RuntimeError("Interactive mode not supported in Codespace")
    
    # Create an interactive plot
    %matplotlib tk
    print("🖱️  Opening interactive polygon drawing tool...")
    print("   Instructions: RIGHT CLICK → add vertex | ENTER → finish polygon | Close window when done")
    
    drawer = InteractivePolygonDrawer(ugrid, nc_path, plot_bathymetry)
    polygons = drawer.draw_polygons()
    
    if polygons:
        output_file = os.path.join(output_dir, 'sandpits.pol')
        save_pol_file(polygons, output_file)
        print(f"✅ Saved {len(polygons)} polygons to {output_file}")


# Uncomment to add one new shape to the existing polygon file
add_shape_to_polygon(input_pol_file, output_pol_file, 'rectangle', 4.0, 52.2, width=0.03, height=0.02, rotation=45)
polygons = load_pol_file(output_pol_file)  # Reload polygons after adding new shape

## Step 2: Plan Refinement Strategy

This step:
1. Analyzes the current grid resolution within the sandpit polygons
2. Calculates the number of refinement steps needed
3. Generates refinement zones with automatic overlap detection and merging
4. Visualizes the refinement plan

The refinement zones are created from coarse (outer) to fine (inner) resolution.

In [ ]:
# Step 2: Plan Refinement Strategy

# Analyze grid resolution and compute refinement steps
print("🔍 Analyzing grid resolution...")
refinement_params = compute_refinement_steps(ugrid, target_resolution, polygons)

# Generate refinement polygons with overlap merging
print("📐 Generating refinement zones...")
(all_refinement_polygons, all_original_polygons, 
 buffer_polygons, expansions) = generate_refinement_polygons(
    polygons, refinement_params, buffer_around_sandpit, N)

# Store original buffer polygons for visualization
original_buffer_polygons = []
for i, polygon in enumerate(polygons):
    polygon_array = np.array(polygon)
    center_lat = np.mean(polygon_array[:, 1])
    expanded_polygon = expand_polygon_outward(polygon, buffer_around_sandpit, center_lat)
    original_buffer_polygons.append(expanded_polygon)

# Visualize refinement plan
if visualization_bool:
    print("📈 Visualizing refinement plan...")
    if not is_codespace():
        %matplotlib inline
    plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
            buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
            original_buffer_polygons=original_buffer_polygons,
            title='Refinement Plan: Sandpit Polygons and Merged Refinement Zones')
    plt.show()

## Step 3: Execute Grid Refinement

This step applies Casulli refinement to the meshkernel object using the generated refinement zones. The refinement is applied from outside to inside (coarse to fine) to ensure smooth transitions.

After refinement, the refined grid is visualized showing the final mesh structure.

In [ ]:
# Step 3: Execute Grid Refinement

# Apply Casulli refinement
print("⚙️  Executing refinement...")
apply_casulli_refinement(mk_object, all_refinement_polygons)

# Visualize refined grid
if visualization_bool:
    print("📊 Visualizing refined grid...")
    if not is_codespace():
        %matplotlib inline
    plot_grid(mk_object, polygons, all_refinement_polygons, all_original_polygons,
            buffer_polygons, refinement_params['envelope_sizes_m'], refinement_params['n_steps'],
            title='Refined Grid: Final Result with Casulli Refinement')

# Print refinement summary
print_refinement_summary(polygons, all_refinement_polygons, 
                        refinement_params['envelope_sizes_m'], 
                        refinement_params['n_steps'], buffer_polygons)

## Step 4: Monitor Grid Quality (Optional)

This optional step analyzes the quality of the refined grid by examining:
- **Resolution**: Face areas converted to characteristic lengths
- **Smoothness**: Ratio of adjacent cell sizes (target < 1.4)
- **Orthogonality**: Deviation from 90° angles (target < 0.01)

**Note**: This analysis can be computationally intensive for large grids, especially in Codespace.

In [ ]:
# Step 4: (Optional) Monitor Grid Quality
if monitor_quality:
    print("🔬 Analyzing grid quality metrics...")
    quality_data = analyze_grid_quality(mk_object, ugrid_original, all_refinement_polygons, polygons)
    
    print("📊 Creating quality visualization...")
    if not is_codespace():
        %matplotlib inline
        plot_grid_quality(quality_data, all_refinement_polygons, target_resolution)
    else:
        print("⚠️  Apparently the kernel crashes when trying to plot this in a Codespace environment...")

## Step 5: Save Refined Grid (Optional)

This step exports the refined grid to a RGFGRID-compatible NetCDF file with automatic versioning to prevent overwriting existing files. The saved file contains complete UGRID topology including all connectivity arrays required by D-Flow FM.

In [ ]:
# Step 5: Export refined grid to NetCDF with proper naming and compatibility

output_refined_nc, ugrid_complete = export_refined_grid(
    mk_object=mk_object,
    original_nc_file=nc_file, 
    output_dir=output_dir,
    suffix="_ref"
)

A. Activities to create new restart file based on new grid:
1. Creating new restart file with correct dimensions / coordinate variables
2. Horizontal interpolation of all 1D parameters (both elements and links)
3. Horizontal interpolation of all 2D parameters (both elements and links)

B. Activities to adapt bathymetry after sandpit excavation
1. Setup parameterization of sandpit shape (input: depth and slope around edges)
2. Lower bed level elevation (FlowElem_bl) & compute excavated volume (before vs after)
3. Adapt 1D parameters based on new bed level if necessary (waterdepth, ...?)
4. Adapt 2D parameters (primarily vertical profile) on new bed level (flow velocities, turbulence, temperature)

C. Fix the partitioning:
1. Modify the starting indices and counts (e.g., partitions_face_start) of the partititions based new cell count


In [ ]:
# # Step 6a: Export *.xyz file for DFM setup:

# # Get coordinates from the refined grid
# refined_grid_coords = mk_object.mesh2d_get()

# # Get bed level from the restart file
# restart_ds = xr.open_dataset(restart_path, decode_timedelta=False)
# restart_bed_levels = restart_ds.FlowElem_bl.values[0,:]

# # Get coordinates
# restart_coords = np.column_stack([restart_ds.FlowElem_xcc.values, restart_ds.FlowElem_ycc.values])
# refined_coords = np.column_stack([refined_grid_coords.node_x, refined_grid_coords.node_y])

# # Interpolation
# bed_levels_interp = scipy.interpolate.griddata(restart_coords, restart_bed_levels, refined_coords, method='linear')
# bed_levels_interp_nearest = scipy.interpolate.griddata(restart_coords, restart_bed_levels, refined_coords, method='nearest')
# bl_nan = np.isnan(bed_levels_interp)
# bed_levels_interp[bl_nan] = bed_levels_interp_nearest[bl_nan]

# # 4. Write to *.xyz file
# np.savetxt(xyz_output_path, np.column_stack([refined_grid_coords.node_x, refined_grid_coords.node_y, bed_levels_interp]), fmt='%.8f')

In [ ]:
# # Step 6b: Load xyz file and dig sandpits with slope transitions

# transition_distance = dig_depth / slope  # in meters

# # Convert buffer distance from meters to degrees (approximate at ~52°N latitude)
# lat_approx = 52.0
# meters_per_degree_lat = 111320
# meters_per_degree_lon = 111320 * np.cos(np.radians(lat_approx))
# meters_per_degree_avg = (meters_per_degree_lat + meters_per_degree_lon) / 2
# transition_distance_deg = transition_distance / meters_per_degree_avg

# # Load xyz file
# xyz_data = np.loadtxt(xyz_output_path)
# x_coords, y_coords, bed_levels = xyz_data[:, 0], xyz_data[:, 1], xyz_data[:, 2].copy()

# # Convert sandpit polygons to shapely and fix invalid polygons
# sandpit_polys = []
# for i, poly_coords in enumerate(polygons):
#     poly = Polygon(poly_coords)
#     if not poly.is_valid or poly.area == 0:
#         poly = poly.buffer(0)  # Fix self-intersections
#         if poly.area == 0:
#             from shapely.geometry import MultiPoint
#             hull = MultiPoint(poly_coords).convex_hull
#             if hasattr(hull, 'area') and hull.area > 0:
#                 poly = hull
#     sandpit_polys.append(poly)

# # Combine polygons
# combined_poly = sandpit_polys[0] if len(sandpit_polys) == 1 else gpd.GeoSeries(sandpit_polys).union_all()
# grid_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(x_coords, y_coords)])

# # Identify excavation zones (transition goes INWARD)
# within_sandpit = grid_points.geometry.within(combined_poly)
# inner_poly = combined_poly.buffer(-transition_distance_deg)  # Negative buffer = inward
# within_inner = grid_points.geometry.within(inner_poly) if hasattr(inner_poly, 'area') and inner_poly.area > 0 else np.zeros(len(grid_points), dtype=bool)
# in_transition = within_sandpit & (~within_inner)
# in_full_excavation = within_inner

# # Apply smooth slope transition
# bed_levels_modified = bed_levels.copy()
# if in_transition.any():
#     # Get and densify boundary coordinates for better distance calculation
#     boundary = combined_poly.boundary
#     if boundary.geom_type == 'LineString':
#         boundary_length = boundary.length
#         num_points = max(100, int(boundary_length / transition_distance_deg * 10))
#         distances = np.linspace(0, boundary_length, num_points)
#         boundary_coords = np.array([boundary.interpolate(d).coords[0] for d in distances])
#     else:
#         boundary_coords_list = []
#         for geom in boundary.geoms:
#             geom_length = geom.length
#             num_points = max(50, int(geom_length / transition_distance_deg * 10))
#             distances = np.linspace(0, geom_length, num_points)
#             geom_coords = np.array([geom.interpolate(d).coords[0] for d in distances])
#             boundary_coords_list.append(geom_coords)
#         boundary_coords = np.vstack(boundary_coords_list)
    
#     # Calculate distances and apply gradual depth reduction
#     transition_points = np.column_stack([x_coords[in_transition], y_coords[in_transition]])
#     distances_deg = cdist(transition_points, boundary_coords)
#     min_distances_deg = np.min(distances_deg, axis=1)
#     min_distances_m = min_distances_deg * meters_per_degree_avg
    
#     # Points at boundary get minimal excavation, points further inward get more
#     depth_reduction = dig_depth * (min_distances_m / transition_distance)
#     depth_reduction = np.clip(depth_reduction, 0, dig_depth)
#     bed_levels_modified[in_transition] -= depth_reduction

# # Apply full depth in inner excavation areas
# bed_levels_modified[in_full_excavation] -= dig_depth

# # Save modified xyz file
# output_path = xyz_output_path.replace('.xyz', '_sandpits_dug.xyz')
# np.savetxt(output_path, np.column_stack([x_coords, y_coords, bed_levels_modified]), fmt='%.8f')

# # Calculate estimated volume change
# depth_changes = bed_levels - bed_levels_modified
# excavated_points = depth_changes > 0
# cell_area_m2 = target_resolution ** 2
# total_excavated_volume_m3 = np.sum(depth_changes[excavated_points]) * cell_area_m2
# total_excavated_volume_Mm3 = total_excavated_volume_m3 / 1_000_000  # Convert to million m³

# print(f"Sandpits dug: {in_full_excavation.sum()} points @ {dig_depth}m, {in_transition.sum()} transition points")
# print(f"Estimated excavated volume: {total_excavated_volume_Mm3:.2f} Mm³ (grid interpolation needed for actual volume)")
# print(f"Saved: {output_path}")

In [ ]:
import os
import shutil
import subprocess
import xml.etree.ElementTree as ET
from pathlib import Path
import re

# =============================================================================
# 7a: Set DFM model and installation paths
# =============================================================================

os.chdir(project_root + "/notebooks")  # Ensure working directory is notebooks

# Model paths (relative)
model_dir = Path("../data/model/B04_2018_coldstart")
original_mdu = model_dir / "DCSM-FM_0_5nm.mdu"
dimr_config = model_dir / "dimr_config.xml"
output_refined_nc = Path(output_refined_nc)

# Installation paths (absolute)
dflowfm_exe = Path("p:/11211460-msc-gw-coupling/02_engines/2025.01/x64/bin/run_dflowfm.bat")
dimr_parallel_exe = Path("p:/11211460-msc-gw-coupling/02_engines/2025.01/x64/bin/run_dimr_parallel.bat")

# Partitioning settings
n_partitions = 10

print(f"Model directory: {model_dir.resolve()}")
print(f"Original MDU: {original_mdu}")
print(f"Refined grid: {output_refined_nc}")

# =============================================================================
# 7b: Create temporary *.mdu-file with short run-time, high restart-output and correct netcdf
# =============================================================================

# Create temporary MDU file
temp_mdu = model_dir / "DCSM-FM_0_5nm_temp.mdu"
shutil.copy(original_mdu, temp_mdu)

print(f"Creating temporary MDU file: {temp_mdu}")

# Read and modify MDU file
with open(temp_mdu, 'r') as f:
    mdu_content = f.read()

# Modify key parameters
modifications = {
    'NetFile': output_refined_nc.name,  # Use refined grid
    'Stopdatetime': '20190127001000',  # Short 10 minute run
    'MapInterval': '600.',  # Map output every 10 minutes
    'RstInterval': '600. 0. 600.',  # Restart output every hour
    'HisInterval': '3600.',  # History output every hour
    'DtUser': '300.',  # 5-minute forcing update
    'DtMax': '60.',  # Max 60s timestep
}

# Apply modifications to MDU content
mdu_lines = mdu_content.split('\n')
for i, line in enumerate(mdu_lines):
    for key, value in modifications.items():
        if line.strip().startswith(key) and '=' in line:
            mdu_lines[i] = f"{key:<40} = {value:<60} # Modified for partitioning test"
            print(f"Modified: {key} = {value}")

# Write modified MDU
with open(temp_mdu, 'w') as f:
    f.write('\n'.join(mdu_lines))

# Copy refined grid to model directory
refined_grid_local = model_dir / output_refined_nc.name
if not refined_grid_local.exists():
    print("Copying refined grid to model directory...")
    shutil.copy(output_refined_nc, refined_grid_local)

# =============================================================================
# 7c: Adjust DIMR config based on number of partitions
# =============================================================================

print(f"Modifying DIMR config for {n_partitions} partitions...")

# Read original XML as text and modify it directly to avoid namespace issues
with open(dimr_config, 'r', encoding='iso-8859-1') as f:
    xml_content = f.read()

# Find and replace the process element content
process_pattern = r'(<process>)[^<]*(</process>)'
process_list = ' '.join(str(i) for i in range(n_partitions))
new_process = f'\\g<1>{process_list}\\g<2>'
xml_content = re.sub(process_pattern, new_process, xml_content)

# Also update the inputFile to use the temporary MDU
xml_content = xml_content.replace('DCSM-FM_0_5nm.mdu', temp_mdu.name)

print(f"Updated process element: {process_list}")
print(f"Updated inputFile: {temp_mdu.name}")

# Save modified XML
temp_dimr_config = model_dir / "dimr_config_temp.xml"
with open(temp_dimr_config, 'w', encoding='iso-8859-1') as f:
    f.write(xml_content)

# =============================================================================
# 7d: Start partitioning the refined grid (and *.mdu) through batch
# =============================================================================

print(f"\nStarting partitioning into {n_partitions} domains...")

# Change to model directory for partitioning
os.chdir(model_dir)

# Build partitioning command
partition_cmd = [
    str(dflowfm_exe),
    f'--partition:ndomains={n_partitions}:icgsolver=6',
    str(temp_mdu.name)
]

print(f"Partition command: {' '.join(partition_cmd)}")

# Run partitioning
try:
    result = subprocess.run(partition_cmd, capture_output=True, text=True, shell=True)
    print("Partitioning output:")
    print(result.stdout)
    if result.stderr:
        print("Partitioning errors:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✓ Partitioning completed successfully")
        
        # Check if partition files were created
        partition_files = list(Path('.').glob(f"{temp_mdu.stem}_0*{temp_mdu.suffix}"))
        print(f"Created {len(partition_files)} partition MDU files:")
        for pf in sorted(partition_files):
            print(f"  - {pf}")
            
    else:
        print(f"✗ Partitioning failed with return code {result.returncode}")
        
except Exception as e:
    print(f"✗ Error during partitioning: {e}")

# =============================================================================
# 7e: Generate restart files by running the model
# =============================================================================

print("\nStarting parallel model run to generate restart files...")

# Build parallel run command
parallel_cmd = [
    str(dimr_parallel_exe),
    str(n_partitions),
    str(temp_dimr_config.name)
]

print(f"Parallel run command: {' '.join(parallel_cmd)}")

# Run parallel model
try:
    result = subprocess.run(parallel_cmd, capture_output=True, text=True, shell=True)
    print("Model run output:")
    print(result.stdout)
    if result.stderr:
        print("Model run errors:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✓ Model run completed successfully")
        
        # Check for restart files
        rst_files = list(Path('.').glob("*_rst.nc"))
        print(f"Generated {len(rst_files)} restart files:")
        for rf in sorted(rst_files):
            print(f"  - {rf} ({rf.stat().st_size / 1e6:.1f} MB)")
            
    else:
        print(f"✗ Model run failed with return code {result.returncode}")
        
except Exception as e:
    print(f"✗ Error during model run: {e}")

print("\nPartitioning and initialization workflow completed!")

os.chdir(project_root + "/notebooks")  # Ensure working directory is notebooks

In [ ]:
# # Step 7: Create New Restart File with Refined Grid

# from scipy.spatial import cKDTree

# # Load and analyze original restart file
# original_restart_ds = xr.open_dataset(restart_path, decode_timedelta=False)

# # Check for duplicate coordinates in original grid
# grid_coords = np.column_stack([ugrid_original.mesh2d_face_x.values, ugrid_original.mesh2d_face_y.values])
# unique_coords, unique_indices = np.unique(grid_coords, axis=0, return_index=True)
# duplicates = len(grid_coords) - len(unique_coords)
# print(f"Grid duplicates: {duplicates:,} duplicate coordinates found")

# # Load and combine all dry points
# dry_files = [r"c:\Users\weste_bt\GitHub\orelse-sandpit-automation\data\input\dry_points\RW_DryPoints.xyz",
#             r"c:\Users\weste_bt\GitHub\orelse-sandpit-automation\data\input\dry_points\Outer_DryPoints.xyz", 
#             r"c:\Users\weste_bt\GitHub\orelse-sandpit-automation\data\input\dry_points\mv2.xyz",
#             r"c:\Users\weste_bt\GitHub\orelse-sandpit-automation\data\input\dry_points\Holland_DryPoints.xyz"]

# dry_points = np.vstack([np.loadtxt(f)[:, :2] for f in dry_files])
# print(f"Loaded {len(dry_points):,} dry points")

# # Filter grid coordinates (keep elements >5m from dry points)
# # 5m ≈ 0.00005 degrees at ~52°N latitude (Netherlands)
# threshold_degrees = 0.00005
# dry_tree = cKDTree(dry_points)
# distances, _ = dry_tree.query(grid_coords)
# wet_mask = distances > threshold_degrees
# wet_grid_coords = grid_coords[wet_mask]
# dry_filtered = (~wet_mask).sum()

# print(f"Dry filtering: {dry_filtered:,} grid points within {threshold_degrees:.6f}° of dry points")

# # Find unused wet grid elements
# restart_coords = np.column_stack([original_restart_ds.FlowElem_xcc.values, original_restart_ds.FlowElem_ycc.values])
# wet_tree = cKDTree(wet_grid_coords)
# _, used_indices = wet_tree.query(restart_coords)
# unused_mask = np.ones(len(wet_grid_coords), dtype=bool)
# unused_mask[used_indices] = False

# # Save unused elements
# unused_coords = wet_grid_coords[unused_mask]
# if len(unused_coords) > 0:
#    np.savetxt('restart_coords.xyz', np.column_stack([restart_coords, 0*np.ones(len(restart_coords))]), fmt='%.6f')
#    np.savetxt('restart_unused.xyz', np.column_stack([unused_coords, 2*np.ones(len(unused_coords))]), fmt='%.6f')
#    np.savetxt('ugrid_coords.xyz', np.column_stack([grid_coords, 1*np.ones(len(grid_coords))]), fmt='%.6f')
#    np.savetxt('ugrid_wet_coords.xyz', np.column_stack([wet_grid_coords, 3*np.ones(len(wet_grid_coords))]), fmt='%.6f')

# print(f"Grid (all): {len(grid_coords):,} \nGrid (wet-only): {len(wet_grid_coords):,}\nRestart (all): {len(restart_coords):,}\nGrid (unused): {len(unused_coords):,}")


# # Plotting grid mesh and face-coordinates (of both grid and restart files)
# fig, ax = plt.subplots(figsize=(16, 12))
# mk_backup.mesh2d_get().plot_edges(ax=ax, linewidth=0.5, color='black', alpha=0.3)

# ax.scatter(grid_coords[:, 0], grid_coords[:, 1], marker='o', s=80, color='k', label='Input grid - all (*_net.nc)', alpha=0.3, linewidths=0., edgecolors='w')
# ax.scatter(restart_coords[:, 0], restart_coords[:, 1], marker='o', s=15, color='tab:red', label='Restart - all (*_rst.nc)', alpha=1)

# ax.set_xlim(3.55, 3.8)
# ax.set_ylim(53.85, 54.00)
# ax.set_aspect('equal')

# ax.legend(loc='upper left', fontsize=12)

# plt.show()

# # # Create analysis structure for comparison
# # variable_info = {}
# # for var_name, var in original_restart_ds.data_vars.items():
# #     var_size_mb = var.size * var.dtype.itemsize / (1024**2)
# #     variable_info[var_name] = {
# #         'shape': var.shape,
# #         'dims': var.dims,
# #         'dtype': str(var.dtype),
# #         'size_mb': var_size_mb,
# #         'attributes': dict(var.attrs)
# #     }

# # restart_analysis = {
# #     'dataset': original_restart_ds,
# #     'file_size_gb': file_size_gb,
# #     'variable_info': variable_info
# # }

# # # Create new restart file with all variables
# # print("🔧 Creating new restart file with all variables...")
# # new_restart_template = create_restart_file(ugrid_complete, restart_analysis)

# # # After creating the restart file template
# # regrid_restart_data(original_restart_ds, new_restart_template, ugrid_complete, ugrid_original)

# # # Compare old and new restart files
# # compare_restart_files(restart_analysis, new_restart_template)